In [1]:
strain = "BY4741"
paralog = "FKS1"
# Chain identifier in the PDB
chain = '/F'

hotspots = ["HS1", "HS2", "HS3"]
compound = "caspofungin"

In [2]:
# Parameters
strain = "BY4741"
paralog = "FKS1"
chain = "/F"
hotspots = ["HS1", "HS2", "HS3"]
compound = "caspofungin"


# Notebook to generate defattr files for ChimeraX

These files are later imported in ChimeraX to plot DMS data on protein structures

## Import libraries

In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

## Enter constants

In [4]:
# Color palette
palette = '#009B9E:#42B7B9:#A7D3D4:#F1F1F1:#E4C1D9:#D691C1:#C75DAB'

## Retrieve all classified data

In [5]:
list_df = []

for hs in hotspots:
    locus = "-".join([paralog, hs])
    hs_df = pd.read_csv(f"../heatmap_data/{strain}_{locus}_{compound}.csv")
    hs_df["Hotspot"] = hs
    list_df.append(hs_df)

df = pd.concat(list_df, ignore_index=True)

## Aggregate

In [6]:
attrdf = df.groupby("aa_pos")[["sensres"]].agg(lambda x: x[x.str.contains('resistant')].count() / len(x)).reset_index()
attrdf['pos'] = attrdf["aa_pos"].apply(lambda x: f'{chain}:{str(int(x))}')
attrdf['empty_col'] = ''
attrdf

,aa_pos,sensres,pos,empty_col
0,639,0.857143,/F:639,
1,640,0.142857,/F:640,
2,641,0.263158,/F:641,
3,642,0.285714,/F:642,
4,643,0.857143,/F:643,
5,644,0.285714,/F:644,
6,645,0.200000,/F:645,
7,646,0.904762,/F:646,
8,647,0.850000,/F:647,
9,690,0.266667,/F:690,


## Convert to defattr file

In [7]:
custom_header = f"#\nattribute: res\nmatch mode: 1-to-1\nrecipient: residues\n"
with open(f"../defattr/{strain}_{paralog}_{compound}_defattr.txt", 'w') as fp:
    fp.write(custom_header)

attrdf[['empty_col','pos','sensres']].to_csv(f"../defattr/{strain}_{paralog}_{compound}_defattr.txt", sep='\t', header=None, index=None, mode='a')

cmd = f'color byattribute res range 0, 1 palette {palette}'
print("Command lines to enter in ChimeraX")
print(f"open \"../defattr/{strain}_{paralog}_{compound}_defattr.txt\" format defattr")
print(cmd)

Command lines to enter in ChimeraX
open "../defattr/BY4741_FKS1_caspofungin_defattr.txt" format defattr
color byattribute res range 0, 1 palette #009B9E:#42B7B9:#A7D3D4:#F1F1F1:#E4C1D9:#D691C1:#C75DAB


In [8]:
# Example of custom color key
print(f'key {palette.split(":")[0]}:Scer '+ ' '.join([f'{c}:' for c in palette.split(':')[1:-1]]) + f' {palette.split(":")[-1]}:Cneo')

key #009B9E:Scer #42B7B9: #A7D3D4: #F1F1F1: #E4C1D9: #D691C1: #C75DAB:Cneo
